In [2]:
import os
import pandas as pd
import sys
sys.path.append('../dependencias')

from conn_spark import CriarConn

In [3]:
spark = CriarConn(name_app = "raw").cria_conn()

your 131072x1 screen size is bogus. expect trouble
25/03/02 12:40:52 WARN Utils: Your hostname, DESKTOP-RE2QNUL resolves to a loopback address: 127.0.1.1; using 172.21.253.66 instead (on interface eth0)
25/03/02 12:40:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 12:40:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
path_arquivo = f'../datasets/prata/GERACAO_USINA/GERACAO_USINA.parquet'
df = spark.read.parquet(path_arquivo)

In [5]:
df.createOrReplaceTempView("tb_geracao_usina")

In [6]:
df.show()

+-------------------+-------------+--------------+---------+----------+----------------------+-------------+-------------------+--------------------+------+--------------------+------------+
|       din_instante|id_subsistema|nom_subsistema|id_estado|nom_estado|cod_modalidadeoperacao|nom_tipousina|nom_tipocombustivel|           nom_usina|id_ons|                 ceg| val_geracao|
+-------------------+-------------+--------------+---------+----------+----------------------+-------------+-------------------+--------------------+------+--------------------+------------+
|2020-12-31 21:00:00|            N|         NORTE|       AM|  AMAZONAS|                TIPO I|HIDROELÉTRICA|         Hidráulica|             Balbina|  AMBA|UHE.PH.AM.000190-...|106.55400000|
|2020-12-31 21:00:00|            N|         NORTE|       AM|  AMAZONAS|              TIPO III|      TÉRMICA|   Óleo Combustível|            Electron| AMELC|UTE.PE.AM.027120-...|        0E-8|
|2020-12-31 21:00:00|            N|         N

In [7]:
df_tratado = spark.sql("""
            SELECT 
                din_instante as dt_referencia,
                id_subsistema as sigla_regiao,
                nom_subsistema as nm_regiao,
                id_estado as sigla_estado,
                nom_estado as nm_estado,
                nom_tipousina as nm_tipoUsina,
                nom_tipocombustivel as nm_tipoCombustivel,
                nom_usina as nm_usina,
                CAST(val_geracao as DECIMAL(10,2)) as vl_energiaGeradaMWmed
            FROM tb_geracao_usina
            WHERE id_estado = 'SP' AND val_geracao <> '0E-8' 
                    """)

In [16]:
df_sum_ano = spark.sql("""
            SELECT YEAR(dt_referencia),COUNT(*),MAX(vl_energiaGeradaMWmed), MIN(vl_energiaGeradaMWmed)
            FROM(SELECT 
                din_instante as dt_referencia,
                id_subsistema as sigla_regiao,
                nom_subsistema as nm_regiao,
                id_estado as sigla_estado,
                nom_estado as nm_estado,
                nom_tipousina as nm_tipoUsina,
                nom_tipocombustivel as nm_tipoCombustivel,
                nom_usina as nm_usina,
                CAST(val_geracao as DECIMAL(10,2)) as vl_energiaGeradaMWmed
            FROM tb_geracao_usina
            WHERE id_estado = 'SP' AND val_geracao <> '0E-8') 
            GROUP BY ALL
            ORDER BY 1 ASC
                    """)

In [17]:
df_sum_ano_mes = spark.sql("""
            SELECT 
                YEAR(dt_referencia) AS ano, 
                MONTH(dt_referencia) as mes,
                COUNT(*) as total_registros,
                MAX(vl_energiaGeradaMWmed) as max_energiaGeradaMWmed,
                MIN(vl_energiaGeradaMWmed) as min_energiaGeradaMWmed
            FROM(SELECT 
                din_instante as dt_referencia,
                id_subsistema as sigla_regiao,
                nom_subsistema as nm_regiao,
                id_estado as sigla_estado,
                nom_estado as nm_estado,
                nom_tipousina as nm_tipoUsina,
                nom_tipocombustivel as nm_tipoCombustivel,
                nom_usina as nm_usina,
                CAST(val_geracao as DECIMAL(10,2)) as vl_energiaGeradaMWmed
            FROM tb_geracao_usina
            WHERE id_estado = 'SP' AND val_geracao <> '0E-8') 
            GROUP BY ALL
            ORDER BY 1,2 ASC
                    """)

In [10]:
path_gold = '../datasets/gold/'

os.makedirs(path_gold, exist_ok=True)

In [11]:
df_tratado.toPandas().to_csv(f'{path_gold}/GERACAO_ENERGIA.csv')

In [18]:
df_sum_ano.toPandas().to_csv(f'{path_gold}/GERACAO_ENERGIA_ANO.csv')

In [19]:
df_sum_ano_mes.toPandas().to_csv(f'{path_gold}/GERACAO_ENERGIA_MES.csv')

In [21]:
spark.stop()